In [2]:
#Adapted from Ben's preprocessing work pushed on 14 Oct, 2020

import pandas as pd
import string
from datetime import datetime
import unidecode
from word2number import w2n

import warnings
warnings.filterwarnings("ignore")


In [6]:
comments = pd.read_csv('data/train-balanced-sarcasm.csv')

In [7]:
comments.head(10)

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
5,0,"I don't pay attention to her, but as long as s...",only7inches,AskReddit,0,0,0,2016-09,2016-09-02 10:35:08,do you find ariana grande sexy ?
6,0,Trick or treating in general is just weird...,only7inches,AskReddit,1,-1,-1,2016-10,2016-10-23 21:43:03,What's your weird or unsettling Trick or Treat...
7,0,Blade Mastery+Masamune or GTFO!,P0k3rm4s7,FFBraveExvius,2,-1,-1,2016-10,2016-10-13 21:13:55,Probably Sephiroth. I refuse to taint his grea...
8,0,"You don't have to, you have a good build, buy ...",SoupToPots,pcmasterrace,1,-1,-1,2016-10,2016-10-27 19:11:06,What to upgrade? I have $500 to spend (mainly ...
9,0,I would love to see him at lolla.,chihawks,Lollapalooza,2,-1,-1,2016-11,2016-11-21 23:39:12,Probably count Kanye out Since the rest of his...


In [9]:
comments.groupby('label').count()

,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
label,,,,,,,,,
0,505405,505413,505413,505413,505413,505413,505413,505413,505413
1,505368,505413,505413,505413,505413,505413,505413,505413,505413


In [ ]:
comments['created_utc'][0]

## Processing Techniques Applied to the Comments

In [86]:
def year_splitter(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    date = datetime.strptime(date_time, format)
    year = date.year
    
    return f'{year}'

def month_extractor(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    date = datetime.strptime(date_time, format)
    month = date.month
    
    return f'{month:02}' 

def day_extractor(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    date = datetime.strptime(date_time, format)
    day= date.day
    
    return f'{day:02}' 

def hour_extractor(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    date = datetime.strptime(date_time, format)
    hour = date.hour
    
    return f'{hour:02}' 

def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    
    return text

def convert_numbers(text):
    token = text.split()

    final = ''
    for word in range(len(token)):
        try:
            number = w2n.word_to_num(token[word])
            final += str(number)
        except:
            final += token[word]
    
        if (word < len(token)):
            final += ' '
        
    return final

def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    
    return text

def remove_single_word(text):
    token = text.split()
    final = ''
    for word in range(len(token)):
        
        if len(token[word])>1:
            final += token[word]
            
            if (word < len(token)):
                final += ' '
    return final
    
def change_negation_words(text):
    
    negation = [ 'ain', 'aren', "arent", 'couldn',  "couldnt", 'didn', "didnt", 'doesn', "doesnt", "don", "dont", 'hadn', 
                "hadnt", 'hasn', "hasnt", 'haven', "havent", 'isn',  "isnt", 'ma', 'mightn', "mightnt", 'mustn', "mustnt", 
                'needn', "neednt", 'shan', "shant", 'shouldn', "shouldnt", 'wasn', "wasnt", 'weren', "werent", 'won', "wont",
                'wouldn', "wouldnt", "cannot"]
    
    token = text.split()
    final = ''
    for word in range(len(token)):
        if (token[word] not in negation):
            final += token[word]
            
            if (word < len(token)):
                final += ' '
        else:
            final += "not "
    return final  

# removed words: 'not', 'no'
def remove_stopwords(text):
    stop = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "youre", "youve", "youll", "youd", 'your', 
        'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "shes", 'her', 'hers', 'herself', 'it', 
        'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 
        'that', "thatll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 
        'having', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 
        'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
        'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
        'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 
        'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 
        "dont", 'should', "shouldve", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', "theyre"]
    
    token = text.split()
    final = ''
    accept_word_count = 0
    
    for word in range(len(token)):
        if (token[word] not in stop):
            if (accept_word_count!=0):
                final += ' '
            final += token[word]
            accept_word_count += 1
            
    
    return final   

'denden den'

In [34]:
# removing the columns not really needed
comments.drop(['author', 'date'], axis = 1, inplace = True)
comments = comments.dropna()

KeyError: "['author' 'date'] not found in axis"

## Actual Comments

In [87]:
# removing all forms of punctuation, maintains spaces
comments['cleaned comment'] = comments['comment'].str.replace('[{}]'.format(string.punctuation), '')

# removing all upper case letters from each comment
comments['cleaned comment'] = comments['cleaned comment'].str.lower()

# converting accented characters to be standard
comments['cleaned comment'] = comments['cleaned comment'].apply(remove_accented_chars)

# convert numbers to numeric where possible
comments['cleaned comment'] = comments['cleaned comment'].apply(convert_numbers)

# remove numbers from each comment
comments['cleaned comment'] = comments['cleaned comment'].apply(remove_numbers)

# remove the tokens with single word
comments['cleaned comment'] = comments['cleaned comment'].apply(remove_single_word)

# change the negation words from each comment
comments['cleaned comment'] = comments['cleaned comment'].apply(change_negation_words)


# removing stopwords from each comment. Stop words taken from NLTK's default stop word list
# removed words: 'not', 'no'
comments['cleaned comment'] = comments['cleaned comment'].apply(remove_stopwords)

# removing any empty rows just incase a few comments were removed all together
comments = comments.dropna()

In [59]:
# extraving the year, month, day and hour that the comment was made. May be useful in the future
comments['year'] = comments['created_utc'].map(lambda x: year_splitter(x))
comments['month'] = comments['created_utc'].map(lambda x: month_extractor(x))
comments['day'] = comments['created_utc'].map(lambda x: day_extractor(x))
comments['hour'] = comments['created_utc'].map(lambda x: hour_extractor(x))

In [80]:
comments

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,cleaned comment,year,month,day,hour,cleaned parent comment
0,0,NC and NH.,politics,2,-1,-1,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",nc nh,2016,10,16,23,yeahgetargumentidpreferlivedncwell
1,0,You do know west teams play against west teams...,nba,-4,-1,-1,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,know west teams play west teams east teams right,2016,11,01,00,blazersmaverickswestsseednotevencarrygoodenou...
2,0,"They were underdogs earlier today, but since G...",nfl,3,3,0,2016-09-22 21:45:37,They're favored to win.,underdogs earlier today since gronks announcem...,2016,09,22,21,favoredwin
3,0,"This meme isn't funny none of the ""new york ni...",BlackPeopleTwitter,-8,-1,-1,2016-10-18 21:03:47,deadass don't kill my buzz,meme not funny none new york nigga ones,2016,10,18,21,deadassnotkillbuzz
4,0,I could use one of those tools.,MaddenUltimateTeam,6,-1,-1,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,could use tools,2016,12,30,17,yepconfirmsawtoolusemadeboyeasportsmut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,reddit.com,2,2,0,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,...",im sure iran korea technology create pigbirdhu...,2009,04,25,00,nocallingengineeredpathogenreportsvirusbitshu...
1010822,1,"whatever you do, don't vote green!",climate,1,1,0,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...,whatever not vote green,2009,05,14,22,movetypicalrecentdonothingapproachcoemissions...
1010823,1,Perhaps this is an atheist conspiracy to make ...,atheism,1,1,0,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...,perhaps atheist conspiracy make christians loo...,2009,01,11,00,screwdisabledivegotgetchurchtime
1010824,1,The Slavs got their own country - it is called...,worldnews,1,1,0,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...,slavs got country called kosovo,2009,01,23,21,ivealwaysunsettledhearlotjewishpeoplesayisrae...


## Parent comments

In [88]:
# doing the same for the parent comments
# removing punctuation
comments['cleaned parent comment'] = comments['parent_comment'].str.replace('[{}]'.format(string.punctuation), '')

# removing all upper case letters from each comment
comments['cleaned parent comment'] = comments['cleaned parent comment'].str.lower()

# converting accented characters to be standard
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(remove_accented_chars)

# convert numbers to numeric where possible
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(convert_numbers)

# remove numbers from each comment
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(remove_numbers)

# remove the tokens with single word
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(remove_single_word)

# change the negation words from each comment
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(change_negation_words)

# removing stopwords from each comment. Stop words taken from NLTK's default stop word list
comments['cleaned parent comment'] = comments['cleaned parent comment'].apply(remove_stopwords)

# removing any empty rows just incase a few comments were removed all together
comments = comments.dropna()

In [90]:
comments

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,cleaned comment,year,month,day,hour,cleaned parent comment
0,0,NC and NH.,politics,2,-1,-1,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",nc nh,2016,10,16,23,yeah get argument id prefer lived nc well
1,0,You do know west teams play against west teams...,nba,-4,-1,-1,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,do know west teams play west teams east teams ...,2016,11,01,00,blazers mavericks wests seed not even carry go...
2,0,"They were underdogs earlier today, but since G...",nfl,3,3,0,2016-09-22 21:45:37,They're favored to win.,underdogs earlier today since gronks announcem...,2016,09,22,21,favored win
3,0,"This meme isn't funny none of the ""new york ni...",BlackPeopleTwitter,-8,-1,-1,2016-10-18 21:03:47,deadass don't kill my buzz,meme not funny none new york nigga ones,2016,10,18,21,deadass not kill buzz
4,0,I could use one of those tools.,MaddenUltimateTeam,6,-1,-1,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,could use tools,2016,12,30,17,yep confirm saw tool use made boy easportsmut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,reddit.com,2,2,0,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,...",im sure iran korea technology create pigbirdhu...,2009,04,25,00,no calling engineered pathogen reports virus b...
1010822,1,"whatever you do, don't vote green!",climate,1,1,0,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...,whatever do not vote green,2009,05,14,22,move typical recent donothing approach co emis...
1010823,1,Perhaps this is an atheist conspiracy to make ...,atheism,1,1,0,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...,perhaps atheist conspiracy make christians loo...,2009,01,11,00,screw disabledive got get church time
1010824,1,The Slavs got their own country - it is called...,worldnews,1,1,0,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...,slavs got country called kosovo,2009,01,23,21,ive always unsettled hear lot jewish people sa...


In [91]:
# saving processed comments into a csv file
comments.to_csv("data/cleaned-train-balanced-sarcasm.csv")

In [29]:
# saving processed comments into a feather file
comments.reset_index().to_feather('data/cleaned-train-balanced-sarcasm.feather')

In [3]:
comments_feather = pd.read_feather(f'cleaned-train-balanced-sarcasm.feather').drop('index', axis=1)

In [4]:
comments_feather.head()

,label,comment,subreddit,score,ups,downs,created_utc,parent_comment,cleaned comment,cleaned parent comment
0,0,NC and NH.,politics,2,-1,-1,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",nc nh,yeah get argument point id prefer lived nc well
1,0,You do know west teams play against west teams...,nba,-4,-1,-1,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,know west teams play west teams east teams right,blazers mavericks wests seed not even carry go...
2,0,"They were underdogs earlier today, but since G...",nfl,3,3,0,2016-09-22 21:45:37,They're favored to win.,underdogs earlier today since gronks announcem...,theyre favored win
3,0,"This meme isn't funny none of the ""new york ni...",BlackPeopleTwitter,-8,-1,-1,2016-10-18 21:03:47,deadass don't kill my buzz,meme funny none new york nigga ones,deadass kill buzz
4,0,I could use one of those tools.,MaddenUltimateTeam,6,-1,-1,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...,could use one tools,yep confirm saw tool use made boy easportsmut
